## Description 
This example demonstrates how to run GeniERuntime through OneWorkflow. A small dummy script for demonstrating SifIO is also provided. This example uses a parametrized containership model from [GeniE Snack Pack](https://sesam.dnv.com/genie_utils/hullforms/containership.html). *AP* and *FP* are parametrized and input given from a Pandas dataframe as shown below.




In [ ]:
from pathlib import Path
from oneWorkflowToolBox import *
workspaceId = "GeniERuntimeExample"
# local workspace, all results will be put here after local or cloud runs
# location of common files for all analysis, has to be below workspacePath and in the folder names CommonFilesr
root_folder = os.getcwd()
print(root_folder)
workspacePath = Path(root_folder, 'Workspace')
cloud_run = True
#If running locally the code below will also start the local workflow host.
workflow_client = one_workflow_client(workspaceId, workspacePath, cloud_run, tmp=r"c:\OneWorkflowTemp", platform=Platform.Windows)
if (cloud_run):
    workflow_client.login()
workflow_client.upload_common_files(FileOptions(max_size_bytes=524_000,patterns=["**/*.*"], overwrite=True))

In [ ]:
import pandas as pd
from oneWorkflowToolBox import run_workflow_async
from SesamHelpers import *
import shutil
import json
# we must delete existing results locally before generating new results
local__result_path = Path(workspacePath, workflow_client.results_directory)
if os.path.isdir(local__result_path):
    shutil.rmtree(local__result_path)

#parametrized values
df = pd.DataFrame({'AP':  ["0m", "0.5m", "1m"], 'FP': ["150m", "250m", "500m"]})
workUnit = GeniERuntimeTaskCreator("ContainerHull_template.js", df,workflow_client).get_genieruntime_work_unit(cloud_run, workspacePath)
downloadOptions = FileOptions(
    max_size_bytes=11124_000,
    patterns=["**/T1.FEM", "**/*.csv"])
job = workflow_client.create_job(workUnit)

#for debugging only
#job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)
await run_workflow_async(job, workflow_client, downloadOptions)


## Postprocessing
The code below prints out node and element counts for the different models and present the results in a table.


In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import glob

lc = 11
frames = []
for folder in glob.glob(f"{local__result_path}\\*"):
    data = pd.read_csv(Path(folder, "nodeCount.csv"), index_col=0)
    data.index = [folder.split('\\')[-1]]
    frames.append(data)
df = pd.concat(frames)
df=df.rename_axis("Loadcase")
display(df)
